In [1]:
import numpy as np 

import pandas as pd 

!conda install -c conda-forge wikipedia --yes 
import wikipedia as wp

!pip install lxml

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - wikipedia


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    soupsieve-1.9.4            |           py36_0          58 KB  conda-forge
    beautifulsoup4-4.8.1       |           py36_0         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    wikipedia-1.4.0            |             py_2          13 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.3 MB

The following NEW packages will be INSTALLED:

    soupsieve:      1.9.4-py36_0

In [2]:
### Read wikipedia page ###

html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html, header=0)[0]

In [3]:
### Drop Borough with Not Assigned ###

df_assigned = df[df.Borough != "Not assigned"]
df_assigned.reset_index(drop =True, inplace =True)
df_assigned.head(15)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [4]:
### Merge rows with same postcode and set 'Not assigned' neighbourhood to Borough ###
df_assigned = df_assigned.groupby(['Postcode','Borough'], as_index=False).agg(lambda x: ','.join(set(x)))
na_nb_idx =df_assigned['Neighbourhood'] == 'Not assigned'
df_assigned.loc[na_nb_idx, 'Neighbourhood'] =  df_assigned[na_nb_idx].Borough
df_assigned[df_assigned['Postcode'] == 'MA7']

,Postcode,Borough,Neighbourhood


In [5]:
### Show the shape of the dataset ###
df_assigned.shape

(103, 3)

In [6]:
### Add lat and long info for each neighborhood ###
import numpy as np
df_assigned['Latitude'] = np.nan
df_assigned['Longitude'] = np.nan

In [7]:
df_assigned.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",NaN,NaN
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",NaN,NaN
2,M1E,Scarborough,"Morningside,Guildwood,West Hill",NaN,NaN
3,M1G,Scarborough,Woburn,NaN,NaN
4,M1H,Scarborough,Cedarbrae,NaN,NaN


In [8]:
!wget -O GeoCord.csv http://cocl.us/Geospatial_data/

--2019-11-07 00:39:35--  http://cocl.us/Geospatial_data/
Resolving cocl.us (cocl.us)... 169.48.113.194
Connecting to cocl.us (cocl.us)|169.48.113.194|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data/ [following]
--2019-11-07 00:39:35--  https://cocl.us/Geospatial_data/
Connecting to cocl.us (cocl.us)|169.48.113.194|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-11-07 00:39:37--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.26.197, 107.152.27.197
Connecting to ibm.box.com (ibm.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-11-07 00:39:38--  https://ibm.box.com/public/static/9

In [9]:
### Read Geocord CSV data ###
df_cord = pd.read_csv('GeoCord.csv')
df_cord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
### For each postcode in df_assigned, find corresponding coordinates in df_cord and assign it to df_assigned ###
for idx in df_assigned.index:
    cord_idx = df_cord['Postal Code'] == df_assigned.loc[idx, 'Postcode'] 
    df_assigned.at[idx, 'Latitude'] = df_cord.loc[cord_idx, 'Latitude'].values
    df_assigned.at[idx, 'Longitude'] = df_cord.loc[cord_idx, 'Longitude'].values
    
df_assigned.head(15)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside,Guildwood,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Ionview,Kennedy Park,East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile,Clairlea,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside,Scarborough Village West,Cliffcrest",43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West,Birch Cliff",43.692657,-79.264848


In [11]:
import json
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          91 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.50   | 

In [13]:
geolocator = Nominatim(user_agent="toronta_cousera")
address = 'Toronto'
try:
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))
except AttributeError:
    print('Cannot find: {}, will drop index: {}'.format(address, index))

### Create map using latitude and longitude values ###
cn_map = folium.Map(location=[latitude, longitude], zoom_start=11)

### Add markers to map ###
for lat, lng, label in zip(df_assigned['Latitude'], df_assigned['Longitude'], df_assigned['Postcode']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(cn_map)  
    
cn_map

The geograpical coordinates of Toronto are 43.653963, -79.387207.
